# PCA 95% variância 

In [5]:
import os
import pandas as pd
import math
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder

def apply_pca(data, target_column, variance_ratio):
    """
    Aplica PCA para reduzir a dimensionalidade dos dados preservando uma porcentagem da variância.

    Args:
        data (pd.DataFrame): Dataset original.
        target_column (str): Nome da coluna alvo.
        variance_ratio (float): Percentual da variância a ser preservada (0-1).

    Returns:
        pd.DataFrame: Dataset com os componentes principais selecionados e a coluna alvo.
    """
    # Separar variáveis independentes e dependente
    X = data.drop(columns=[target_column])
    y = data[target_column]

    # Identificar variáveis numéricas para padronização
    numeric_columns = X.select_dtypes(include=['number']).columns
    non_numeric_columns = X.select_dtypes(exclude=['number']).columns

    # Padronizar apenas as variáveis numéricas
    scaler = StandardScaler()
    X_scaled = X.copy()
    X_scaled[numeric_columns] = scaler.fit_transform(X[numeric_columns])

    # Aplicar PCA para preservar a variância desejada
    pca = PCA(n_components=variance_ratio)
    X_pca = pca.fit_transform(X_scaled[numeric_columns])

    print(f"Preservando {variance_ratio * 100}% da variância: {pca.n_components_} componentes principais selecionados")

    # Criar novo DataFrame com os componentes principais
    pca_columns = [f"PCA_{i+1}" for i in range(X_pca.shape[1])]
    new_data = pd.DataFrame(X_pca, columns=pca_columns)

    # Adicionar de volta as colunas não numéricas
    new_data = pd.concat([new_data, X[non_numeric_columns].reset_index(drop=True)], axis=1)
    new_data[target_column] = y.values

    return new_data

def get_next_folder_number(output_root):
    """
    Determina o próximo número de pasta disponível no diretório de saída.

    Args:
        output_root (str): Diretório raiz para salvar as pastas processadas.

    Returns:
        int: O próximo número disponível para a pasta de saída.
    """
    existing_folders = [
        folder for folder in os.listdir(output_root) if os.path.isdir(os.path.join(output_root, folder))
    ]
    numbered_folders = [
        int(folder.split("-")[0]) for folder in existing_folders if folder.split("-")[0].isdigit()
    ]
    return max(numbered_folders, default=0) + 1

def process_dataset_in_folder(input_dir, output_dir, variance_ratio):
    """
    Processa os datasets em uma pasta e salva os resultados em outra pasta correspondente.

    Args:
        input_dir (str): Caminho do diretório de entrada.
        output_dir (str): Caminho do diretório de saída.
        variance_ratio (float): Percentual da variância a ser preservada (0-1).
    """
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.endswith(".txt"):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)

            print(f"Processando arquivo: {input_path}")
            try:
                data = pd.read_csv(input_path, delimiter=",", skipinitialspace=True)
                target_column = data.columns[-1]  # Assume que a última coluna é a coluna alvo

                # Aplicar PCA
                new_data = apply_pca(data, target_column, variance_ratio)

                # Salvar o novo dataset
                new_data.to_csv(output_path, sep=",", index=False)
                print(f"Arquivo processado e salvo em: {output_path}")

            except Exception as e:
                print(f"Erro ao processar {filename}: {e}")

def process_multiple_folders(input_root, output_root, variance_ratio):
    """
    Processa múltiplas pastas dentro do diretório de entrada e salva os resultados diretamente
    no diretório de saída com nomes incrementados.

    Args:
        input_root (str): Diretório raiz contendo as pastas de entrada.
        output_root (str): Diretório raiz para salvar as pastas processadas.
        variance_ratio (float): Percentual da variância a ser preservada (0-1).
    """
    next_folder_number = get_next_folder_number(output_root)

    for folder_name in os.listdir(input_root):
        input_folder_path = os.path.join(input_root, folder_name)
        output_folder_name = f"{next_folder_number}-saida"
        output_folder_path = os.path.join(output_root, output_folder_name)

        if os.path.isdir(input_folder_path):
            print(f"\nProcessando pasta {folder_name} para {output_folder_name}...")
            process_dataset_in_folder(input_folder_path, output_folder_path, variance_ratio)
            next_folder_number += 1

# Diretórios de entrada e saída
input_root_directory = r"C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\proposal\\algorithms\\abordagens\\output"
output_root_directory = r"C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\proposal\\algorithms\\abordagens\\output4"

# Processar arquivos usando PCA para preservar 95% da variância
top_variance_ratio = 0.95  # 95% da variância
process_multiple_folders(input_root_directory, output_root_directory, variance_ratio=top_variance_ratio)



Processando pasta 0-saida para 1-saida...
Processando arquivo: C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\proposal\\algorithms\\abordagens\\output\0-saida\cocomo81.txt
Preservando 95.0% da variância: 12 componentes principais selecionados
Arquivo processado e salvo em: C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\proposal\\algorithms\\abordagens\\output4\1-saida\cocomo81.txt
Processando arquivo: C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\proposal\\algorithms\\abordagens\\output\0-saida\encoded_china.txt
Preservando 95.0% da variância: 11 componentes principais selecionados
Arquivo processado e salvo em: C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\proposal\\algorithms\\abordagens\\output4\1-saida\encoded_china.txt
Processando arquivo: C:\\Users\\CALEO\\OneDrive - Hexagon\\Documents\\GitHub\\Software_effort_estimation\\propo